In [1]:
import csv
import numpy as np
import variables
import cPickle as pkl

import spacy
nlp = spacy.load('fr')



    Only loading the 'fr' tokenizer.



In [ ]:
def load_embeddings():
    dictfn = 'data/word2vec_fr_200'
    
    vocab_size = 0
    with open(dictfn, 'r') as dictfile:
        for _ in dictfile:
            vocab_size += 1
    
    nb_words = vocab_size + 1
    embedding_dim = variables.EMBEDDING_DIM
    embedding_weights = np.zeros((nb_words, embedding_dim))
    ids_dict = {}
    
    with open(dictfn) as w2v:
        word_id = 1
        for line in w2v:
            words = line.split()
            embedding_weights[word_id, :] = map(float, words[1:])
            ids_dict[words[0]] = word_id
            word_id += 1
    
        with open('cache/ids_dictionary.pkl', 'wb') as idsfile:
            pkl.dump(ids_dict, idsfile)
        
        np.save('cache/embedding_weights.npy', embedding_weights)
load_embeddings()

In [2]:
def tokens2vec(tokens, ids_dict):
    ids_vec = np.zeros((1, variables.MAX_LEN))
    
    for idx,token in enumerate(tokens):
        if idx >= variables.MAX_LEN:
            break
        try:
            ids_vec[0,idx] = ids_dict[token.text.lower()]
        except KeyError:
            continue
    return ids_vec

In [6]:
with open('data/output_train.csv','rU') as csvfile:
    filereader = csv.reader(csvfile, delimiter=';')
    y_train = []
    for idx,row in enumerate(filereader):
        if idx == 0:
            continue
        y_train.append(int(row[1]))
    y_train = np.asarray(y_train)[:,np.newaxis]
    np.save('cache/train/labels.npy', y_train)
    
    print y_train.shape

(80000, 1)


In [4]:
with open('data/input_train.csv','rU') as csvfile:
    dictfn = 'cache/ids_dictionary.pkl'
    with open(dictfn) as dictfile:
        ids_dict = pkl.load(dictfile)
        
    filereader = csv.reader(csvfile, delimiter=';')
    
    X_train = np.zeros((0, variables.MAX_LEN))
    title_train = np.zeros((0, variables.MAX_LEN))
    rating_train = []

    for idx,row in enumerate(filereader):
        if idx == 0:
            continue
        if (idx+1) % 1000 == 0 :
            print 'Processing index {}'.format(idx+1)
        
        text = nlp(unicode(row[1], 'utf-8'))
        X_train = np.concatenate((X_train, tokens2vec(text, ids_dict)))
        
        title = nlp(unicode(row[2], 'utf-8'))
        title_train = np.concatenate((title_train, tokens2vec(title, ids_dict)))
        
        rating_train.append(int(row[3]))
    
    np.save('cache/train/data.npy', X_train)
    np.save('cache/train/title.npy', title_train)
    np.save('cache/train/rating.npy', np.asarray(rating_train)[:,np.newaxis])
       
    print X_train.shape
    print title_train.shape
    print np.asarray(rating_train).shape

Processing index 1000
Processing index 2000
Processing index 3000
Processing index 4000
Processing index 5000
Processing index 6000
Processing index 7000
Processing index 8000
Processing index 9000
Processing index 10000
Processing index 11000
Processing index 12000
Processing index 13000
Processing index 14000
Processing index 15000
Processing index 16000
Processing index 17000
Processing index 18000
Processing index 19000
Processing index 20000
Processing index 21000
Processing index 22000
Processing index 23000
Processing index 24000
Processing index 25000
Processing index 26000
Processing index 27000
Processing index 28000
Processing index 29000
Processing index 30000
Processing index 31000
Processing index 32000
Processing index 33000
Processing index 34000
Processing index 35000
Processing index 36000
Processing index 37000
Processing index 38000
Processing index 39000
Processing index 40000
Processing index 41000
Processing index 42000
Processing index 43000
Processing index 440

In [5]:
with open('data/input_test.csv','rU') as csvfile:
    dictfn = 'cache/ids_dictionary.pkl'
    with open(dictfn) as dictfile:
        ids_dict = pkl.load(dictfile)
        
    filereader = csv.reader(csvfile, delimiter=';')
    
    X_test = np.zeros((0, variables.MAX_LEN))
    title_test = np.zeros((0, variables.MAX_LEN))
    rating_test = []

    for idx,row in enumerate(filereader):
        if idx == 0:
            continue
        if (idx+1) % 1000 == 0 :
            print 'Processing index {}'.format(idx+1)
        
        text = nlp(unicode(row[1], 'utf-8'))
        X_test = np.concatenate((X_test, tokens2vec(text, ids_dict)))
        
        title = nlp(unicode(row[2], 'utf-8'))
        title_test = np.concatenate((title_test, tokens2vec(title, ids_dict)))
        
        rating_test.append(int(row[3]))
    
    np.save('cache/test/data.npy', X_test)
    np.save('cache/test/title.npy', title_test)
    np.save('cache/test/rating.npy', np.asarray(rating_test)[:,np.newaxis])
       
    print X_test.shape
    print title_test.shape
    print np.asarray(rating_test).shape

Processing index 1000
Processing index 2000
Processing index 3000
Processing index 4000
Processing index 5000
Processing index 6000
Processing index 7000
Processing index 8000
Processing index 9000
Processing index 10000
Processing index 11000
Processing index 12000
Processing index 13000
Processing index 14000
Processing index 15000
Processing index 16000
Processing index 17000
Processing index 18000
Processing index 19000
Processing index 20000
Processing index 21000
Processing index 22000
Processing index 23000
Processing index 24000
Processing index 25000
Processing index 26000
Processing index 27000
Processing index 28000
Processing index 29000
Processing index 30000
Processing index 31000
Processing index 32000
Processing index 33000
Processing index 34000
Processing index 35000
Processing index 36000
(36395, 90)
(36395, 90)
(36395,)
